In [ ]:
"""Imports"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from pathlib import Path
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics import Accuracy, Precision, Recall, F1Score, ConfusionMatrix
from torchmetrics.classification import MulticlassConfusionMatrix
from torchvision import transforms
from torchvision.io import decode_image

In [ ]:
"""Set seeds and device"""
np.random.seed(73)
torch.manual_seed(73)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(73)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Prepare data for CNN architecture

In [ ]:
"""Custom classes for loading data through dataloader as adapted from https://docs.pytorch.org/tutorials/beginner/data_loading_tutorial.html"""
class SpectrogramDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = Path(self.img_dir, self.img_labels.iloc[idx, 1])
        image = decode_image(img_path)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
"""Load data using custom class defined above and split into train/validate/test datasets"""
DATA_PATH = Path(Path.cwd().parent, "Data")
data_transformers=transforms.Compose([transforms.ToPILImage(),
                                      transforms.ToTensor()])

leaf_dataset = SpectrogramDataset(annotations_file=Path(DATA_PATH, "Spectrogram", "all.csv"), img_dir=Path(DATA_PATH, "Spectrogram"), transform=data_transformers)
train_dataset, valid_dataset, test_dataset = random_split(leaf_dataset, [0.6, 0.2, 0.2])

train_dataset

In [ ]:
"""Convert those datasets into batch loaders"""
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

train_loader

Build Model

In [ ]:
"""Combined CNN-LSTM architecture as adapted from https://www.mathworks.com/help/deeplearning/ug/sequence-classification-using-cnn-lstm-network.html"""
class ComposerCNN(nn.Module):
    def __init__(self, input_size, filter_size, num_filters, num_hidden_units, num_classes):
        super(ComposerCNN, self).__init__()
        self.convolve = nn.Sequential(
            nn.Conv2d(in_channels=input_size, out_channels=num_filters, kernel_size=filter_size, padding="same"),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=num_filters, out_channels=num_filters, kernel_size=filter_size, padding="same"),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 2), stride=(4, 2)),

            nn.Conv2d(in_channels=num_filters, out_channels=2*num_filters, kernel_size=filter_size, padding="same"),
            nn.BatchNorm2d(2*num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 2), stride=(4, 2)),

            nn.Conv2d(in_channels=2*num_filters, out_channels=2*num_filters, kernel_size=filter_size, padding="same"),
            nn.BatchNorm2d(2*num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(4, 2), stride=(4, 2)),

            nn.Flatten(),
            #TODO : Add fully connected layers here
            nn.Softmax()
        )

    def forward(self, x):
        return self.convolve(x)

Train Model

In [ ]:
"""Define training loop for CNN-LSTM model"""
def train(model, training, validation, lr=0.001):
	criterion = nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(model.parameters(), lr=lr)
	
	train_losses = []
	for inputs, labels in training:
		model.train()
		optimizer.zero_grad()
		output = model(inputs)
		loss = criterion(output, labels)
		loss.backward()
		optimizer.step()
		train_losses.append(loss.item())
		break

	validation_losses = []
	with torch.no_grad():
		for v_inputs, v_labels in validation:
			model.eval()
			v_output = model(v_inputs)
			validation_loss = criterion(v_output, v_labels)	
			validation_losses.append(validation_loss.item())
			break
	return train_losses, validation_losses

# Train CNN model
epochs = 300
model = ComposerCNN(input_size=1, filter_size=3, num_filters=64, num_hidden_units=256, num_classes=4)

total_train_losses = []
total_validation_losses = []

for epoch in range(epochs):
	epoch_train_loss, epoch_validation_loss = train(model, train_loader, valid_loader)
	total_train_losses.extend(epoch_train_loss)
	total_validation_losses.extend(epoch_validation_loss)
	if (epoch + 1) % 10 == 0:
		print(f"Epoch {epoch + 1}, train loss: {epoch_train_loss[-1]:.4f}, validation loss: {epoch_validation_loss[-1]:.4f}")

In [ ]:
"""Plot training and Validation losses"""
plt.title("LSTM training losses")
plt.ylabel("Loss")
plt.xlabel("Epoch")

plt.plot(total_train_losses, label="Train loss")
plt.plot(total_validation_losses, label="Validation loss")
plt.legend()
plt.show()

Evaluate Model

In [ ]:
"""Create tensors to use in performance metric creation"""
prediction = torch.Tensor()
truth = torch.Tensor()

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        prediction = torch.cat([prediction, predicted], dim=0)
        truth = torch.cat([truth, labels], dim=0)
        
assert prediction.shape == truth.shape
print(prediction.shape)

In [ ]:
"""Create performance metrics"""
accuracy = Accuracy(task="multiclass", num_classes=32)
precision = Precision(task="multiclass", average="macro", num_classes=32)
recall = Recall(task="multiclass", average="macro", num_classes=32)
f1_score = F1Score(task="multiclass", average="macro", num_classes=32)
confusion_matric = ConfusionMatrix(task="multiclass", num_classes=32)

calculated_accuracy = accuracy(truth, prediction)
calculated_precision = precision(truth, prediction)
calculated_recall = recall(truth, prediction)
calculated_f1_score = f1_score(truth, prediction)
calculated_confusion_matric = confusion_matric(truth, prediction)

print("Model Accuracy:", calculated_accuracy)
print("Model Precision:", calculated_precision)
print("Model Recall:", calculated_recall)
print("Model F1:", calculated_f1_score)

In [ ]:
"""Vizulaize confusion matrix"""
metric = MulticlassConfusionMatrix(num_classes=4)
metric.update(truth, prediction)
fig_, ax_ = metric.plot()

Optimize hyperparameters

In [ ]:
#TODO: play with the parameters and optimize the CNN architecture